# Road Following + Collision Avoidance on Jetbot

In this notebook we will be combining both optimized regression and classification models into one notebook so that our Jetbot will be able to perform *road following* as well as enable *collision avoidance* at the same time so that it can avoid collisions with obstacles that come on its way in real-time.

To begin, let's execute the code below to transfer the device from CPU memory to the GPU device.

In [1]:
import torch
device = torch.device('cuda')
print(torch.__version__)

1.3.0


### **Road Following**   
- Upload the "*best_steering_model_xy_trt.pth*" model file obtained from the "live_demo_build_trt.ipynb" into this notebooks's directory. Once that's finished there should be a file named ``best_steering_model_xy_trt.pth`` in this notebook's directory.    

### **Collision Avoidance**             
- Upload the "*best_model_trt.pth*" model file obtained from the "live_demo_resnet18_build_trt.ipnb" into this notebooks's directory. Once that's finished there should be a file named ``best_model_trt.pth`` in this notebook's directory.   
> **Note**: Collision Avoidance ``blocked`` class should include images of cars or any obstacle which you would expect to see on the road (It's good to start with one object first) meanwhile the ``free`` class should include background images of the road following track. 

Load the TRT optimized models by executing the cell below

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth')) # well trained road following model

model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('best_model_trt.pth')) # well trained collision avoidance model

<All keys matched successfully>

### Creating the Pre-Processing Function

We have now loaded our models, but there's a slight issue. The format that we trained our models on doesn't exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera. You should be pretty familiar with this by now. 

In [4]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=10)

In [5]:
image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

We'll also create our robot instance which we'll need to drive the motors.

In [6]:
from jetbot import Robot

robot = Robot()

Now, we will define some sliders to control the JetBot
> **Note**: We have initialized the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control slider: To start your JetBot increase ``speed_control_slider`` 
2. Steering Gain slider: If you see your JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias slider: If you see your JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

4. Blocked slider: Display the probability in which there is an obstacle in the front of the Jetbot using the collision avoidance model
5. Time for stop slider: To manually set the time for which the jetbot should remain stopped after an object has been removed
6. Blocked threshold slider: To manually set the blocked threshold to stop the Jetbot after an object has been detected 

In [7]:
#Road Following sliders
speed_control_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed control')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.04, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_control_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

#Collision Avoidance sliders
blocked_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')
stopduration_slider= ipywidgets.IntSlider(min=1, max=1000, step=1, value=10, description='time for stop') 
blocked_threshold= ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.8, description='blocked threshold')

display(image_widget)

display(ipywidgets.HBox([blocked_slider, blocked_threshold, stopduration_slider]))

FloatSlider(value=0.0, description='speed control', max=1.0, step=0.01)

FloatSlider(value=0.04, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network models for Road following and Collision Avoidance
3. Check an if statements which would allow the Jetbot to perform road following and stop whenever an obstacle has been detected 
4. Compute the approximate steering value
5. Control the motors using proportional / derivative control (PD)

In [15]:
import math
import PIL.Image
import torchvision.transforms as transforms
import torch.nn.functional as F
import time

angle = 0.0
angle_last = 0.0
count_stops = 0
go_on = 1
stop_time = 10 # The number of frames to remain stopped
speed_value = speed_control_slider.value

def execute(change):
    global angle, angle_last, blocked_slider, robot, count_stops, stop_time, go_on, x, y, blocked_threshold
    global speed_value, steer_gain, steer_dgain, steer_bias
                
    steer_gain = steering_gain_slider.value
    steer_dgain = steering_dgain_slider.value
    steer_bias = steering_bias_slider.value
       
#     image_preproc = preprocess(change['new']).to(device)
     
    #Collision Avoidance model:
    image = change['new']
    # image = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)(image)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.resize(image, (224, 224))
    image = transforms.functional.to_tensor(image)
    
    #preprocessing for model : Road following
    #model : model of road following 
#     x2 = model(torch.unsqueeze(image, 0).to(device).half())
#     out2 = F.softmax(x2, dim = 1)
#     out2 = out.max(1)[1]    
    #=======================
    
    image_preproc = preprocess(change['new']).to(device)
    prob_blocked = float(F.softmax(model_trt_collision(torch.unsqueeze(image_preproc, 0).to(device).half()), dim=1).flatten()[0])
    print(prob_blocked,"prob blocked")
        
    blocked_slider.value = prob_blocked    
    stop_time=stopduration_slider.value
   
    if prob_blocked > 0.4: # threshold should be above 0.5
        print("blocked")
        robot.stop()
        time.sleep(0.2)
    else:
        #start of road following detection
        print("free")
#             xy = model_trt(image_preproc).detach().float().cpu().numpy().flatten()
        x = model_trt(torch.unsqueeze(image, 0).to(device).half())
        out = F.softmax(x, dim = 1)
#             out = out.max(1)[0]
        out = out.max(1)[1]
        print(out)
        speed_value = speed_control_slider.value

        if out == 0:
            robot.forward(0.5)
            time.sleep(0.15)
            robot.stop()
            print('forward')
        elif out == 1:
            robot.left(0.75)
            time.sleep(0.15)
            robot.stop()
            print('left')
        elif out == 2:
            robot.right(0.75)
            time.sleep(0.15)
            robot.stop()
            print('right')
        else:
            print('stop')
            robot.stop()
    time.sleep(0.4)
execute({'new': camera.value})

0.72314453125 prob blocked
blocked
0.72314453125 prob blocked
blocked
0.42041015625 prob blocked
free
tensor([1], device='cuda:0')
left
0.42041015625 prob blocked
free
tensor([1], device='cuda:0')
left
0.42041015625 prob blocked
free
tensor([1], device='cuda:0')
left
0.52490234375 prob blocked
blocked
0.52490234375 prob blocked
free
tensor([2], device='cuda:0')
right
0.52490234375 prob blocked
free
tensor([2], device='cuda:0')
right
0.78955078125 prob blocked
blocked
0.78955078125 prob blocked
blocked
0.78955078125 prob blocked
blocked
0.25732421875 prob blocked
free
tensor([0], device='cuda:0')
forward
0.25732421875 prob blocked
free
tensor([0], device='cuda:0')
forward
0.25732421875 prob blocked
free
tensor([0], device='cuda:0')
forward
0.9423828125 prob blocked
blocked
0.9423828125 prob blocked
blocked
0.9423828125 prob blocked
blocked
0.9423828125 prob blocked
blocked
0.9423828125 prob blocked
blocked
0.9423828125 prob blocked
blocked
0.9423828125 prob blocked
blocked
0.9423828125 

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower and collision avoider should work, but the neural network is only as good as the data it's trained on!

In [14]:
camera.observe(execute, names='value')

0.52099609375 prob blocked
free
tensor([1], device='cuda:0')
left
0.296875 prob blocked
free
tensor([1], device='cuda:0')
left
0.296875 prob blocked
free
tensor([1], device='cuda:0')
left
0.296875 prob blocked
free
tensor([1], device='cuda:0')
left
0.375 prob blocked
free
tensor([1], device='cuda:0')
left
0.375 prob blocked
free
tensor([1], device='cuda:0')
left
0.375 prob blocked
free
tensor([1], device='cuda:0')
left
0.2666015625 prob blocked
free
tensor([1], device='cuda:0')
left
0.2666015625 prob blocked
free
tensor([1], device='cuda:0')
left
0.2666015625 prob blocked
free
tensor([1], device='cuda:0')
left
0.23681640625 prob blocked
free
tensor([2], device='cuda:0')
right
0.23681640625 prob blocked
free
tensor([2], device='cuda:0')
right
0.23681640625 prob blocked
free
tensor([2], device='cuda:0')
right
0.31396484375 prob blocked
free
tensor([2], device='cuda:0')
right
0.31396484375 prob blocked
free
tensor([2], device='cuda:0')
right
0.31396484375 prob blocked
free
tensor([2], dev

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow the track and avoid collisions effectively.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [12]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

0.9453125 prob blocked
blocked
0.9453125 prob blocked
blocked
0.9345703125 prob blocked
blocked
0.94140625 prob blocked
blocked
0.93359375 prob blocked
blocked
0.9228515625 prob blocked
blocked
0.943359375 prob blocked
blocked
0.94921875 prob blocked
blocked
0.625 prob blocked
blocked
0.94970703125 prob blocked
blocked
0.95263671875 prob blocked
blocked
0.955078125 prob blocked
blocked
0.404296875 prob blocked
free
tensor([1], device='cuda:0')
left
0.81591796875 prob blocked
blocked
0.4794921875 prob blocked
free
tensor([0], device='cuda:0')
forward
0.6396484375 prob blocked
blocked
0.3671875 prob blocked
free
tensor([0], device='cuda:0')
forward
0.331787109375 prob blocked
free
tensor([0], device='cuda:0')
forward
0.3486328125 prob blocked
free
tensor([0], device='cuda:0')
forward
0.376953125 prob blocked
free
tensor([0], device='cuda:0')
forward
0.335693359375 prob blocked
free
tensor([1], device='cuda:0')
left
0.3203125 prob blocked
free
tensor([1], device='cuda:0')
left
0.333740234